This script exists to couple powergenome (which sweeps location, year, and grid demand scenario to create different template folders)
with the batch csv caserunner (which sweeps carbon constraint, WEC cost, and WEC power limit to create a different replacement.csv for each location).
Notice that location is swept in both powergenome and caserunner, so this script also covers that interaction.

In [1]:
import subprocess
from pathlib import Path
import os
from juliacall import Main as jl
import subprocess
import os
import sys


In [27]:
# paths
pg_script = "./get-powergenome-data.sh"
pg_output = "./results"

subprocess.run(["bash",pg_script]) # run power genome to generate all template folders

File ‘data_east/misc_gen_inputs.csv’ already there; not retrieving.

File ‘data_east/scenario_inputs.csv’ already there; not retrieving.

File ‘data_east/demand_segments_voll.csv’ already there; not retrieving.

File ‘data_east/resource_capacity_spur.csv’ already there; not retrieving.

File ‘data_east/emission_policies.csv’ already there; not retrieving.

File ‘data_east/Reserves.csv’ already there; not retrieving.

File ‘data_CA/test_scenario_inputs.csv’ already there; not retrieving.

File ‘data_CA/test_demand_segments_voll.csv’ already there; not retrieving.

File ‘data_CA/test_misc_gen_inputs.csv’ already there; not retrieving.

06:33:45 [    INFO] powergenome:180 Reading settings file
06:33:45 [ WARNING] powergenome.util:446 
    *****************************
    The ATB technology "OffShoreWind_OTRG10" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string

Using PUDL_DB at sqlite:///data/pudl/pudl.sqlite
Using PG_DB at sqlite:///data/pg/_pg_misc_tables.sqlite3
about to connect to PG database


06:33:48 [ WARNING] powergenome.eia_opendata:196 EIA fuel region names were not found in the settings ('eia_series_region_names'). Applying default values.
06:33:48 [ WARNING] powergenome.eia_opendata:218 EIA fuel names were not found in the settings ('eia_series_fuel_names'). Applying default values:

{'coal': 'STC', 'naturalgas': 'NG', 'distillate': 'DFO', 'uranium': 'U'}
06:33:49 [    INFO] powergenome.generators:3166 Compiling existing generators and their attributes
2025-06-20 06:33:50 [    INFO] catalystcoop.pudl.transform.eia861:456 Started with 336 missing BA Codes out of 11014 records (3.05%)
2025-06-20 06:33:51 [    INFO] catalystcoop.pudl.transform.eia861:480 Ended with 336 missing BA Codes out of 11014 records (3.05%)
2025-06-20 06:33:51 [    INFO] catalystcoop.pudl.output.eia860:177 96.9% of plant records have consistently reported BA Codes
2025-06-20 06:33:51 [    INFO] catalystcoop.pudl.output.eia860:227 Before any filling treatment has been applied. 3.1% of records have

Using PUDL_DB at sqlite:///data/pudl/pudl.sqlite
Using PG_DB at sqlite:///data/pg/pg_misc_tables_efs_2023.2.sqlite
about to connect to PG database


06:34:07 [ WARNING] powergenome.util:446 
    *****************************
    The ATB technology "OffShoreWind_OTRG10" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practice to
    include the full name in this format. Check your settings file.
        
06:34:07 [ WARNING] powergenome.util:446 
    *****************************
    The ATB technology "UtilityPV_Chicago" listed in your settings file under 'atb_new_gen'
    is not fully specified in the 'cost_multiplier_technology_map' settings parameter.
    Part of the <tech>_<tech_detail> string might be included, but it is best practice to
    include the full name in this format. Check your settings file.
        
06:34:07 [    INFO] powergenome.util:534 The default interest compounding method for calculating annuities has changed from continuous to discrete. This m

CompletedProcess(args=['bash', './get-powergenome-data.sh'], returncode=0)

In [ ]:
import sqlalchemy as sa
pg_db = 'sqlite:////home/becca/Documents/git/WEC-DECIDER/modules/CEM/data/pg/_pg_misc_tables.sqlite3'
#pg_db = 'sqlite:///data/pg/_pg_misc_tables.sqlite3'
pg_engine = sa.create_engine(pg_db, pool_pre_ping=True)
c = pg_engine.connect()

In [7]:

!pwd
# cd to WEC-DECIDER
#os.chdir('./../..')
os.chdir('modules/CEM')
!pwd


/home/becca/Documents/git/WEC-DECIDER
/home/becca/Documents/git/WEC-DECIDER/modules/CEM


In [ ]:

# locations
# locs =["CAISO"]
locs =["CAISO"]
pg_id = 0

for loc in locs:
    template_folders = Path(pg_output).glob('**/'+loc)
    #print(list(template_folders))
    for template_folder in template_folders:
        print(template_folder)
        # jl.seval("""cd("modules/CEM")""")
        jl.seval("""Pkg.instantiate()""")
        jl.seval("""Pkg.status()""")
        jl.seval("""Pkg.project()""")
        
        jl.seval("""include("case_runner_new.jl")""")
        jl.seval("""run_caserunner(" """ + str(pg_id) + """ ", " """ + loc + """ ")""")
        pg_id += 1
        # run caserunner with the replacement.csv which corresponds to the proper location.
    

In [3]:

print("Template folders found for", loc, ":", list(template_folders))

Template folders found for CAISO : []


The script below is running 3rd party caserunner. To run this file, uncomment 
include("/Users/kuankhaixin/WEC-DECIDER/modules/CEM/caserunner_v1.jl") in Run.jl. 

In [ ]:

# Determine the root directory of the project 
if "__file__" in globals():
    # Script context
    script_dir = os.path.dirname(os.path.abspath(__file__))
else:
    script_dir = os.getcwd()


root_dir = os.path.abspath(os.path.join(script_dir, "..", "..")) 
cem_dir = os.path.join(root_dir, "modules", "CEM")
case_folder = "case_1_0"
run_file_path = os.path.join(cem_dir, "Cases", case_folder, "Run.jl")
julia_project_path = cem_dir  

# Check that Run.jl exists
if not os.path.exists(run_file_path):
    print(f" Error: Run.jl not found at {run_file_path}")
    sys.exit(1)

# Launch Julia
print(f" Running GenX for case: {case_folder}...")

process = subprocess.Popen(
    ["julia", f"--project={julia_project_path}", run_file_path],
    cwd=root_dir,
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,  # Merge stderr into stdout
    text=True,
    bufsize=1  # Line-buffered
)

# Stream all output
for line in process.stdout:
    print(line, end="")

process.wait()



Running GenX for case: case_1_0...
  Activating project at `~/WEC-DECIDER/modules/CEM`
    Building Gurobi → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/0af671809e0cde0131a2f8607397197fd3648084/build.log`
Case 1 now creating
Case 2 now creating
Case 3 now creating
Case 4 now creating
Case 5 now creating
Case 6 now creating
Case 7 now creating
Case 8 now creating
Case 9 now creating
Case 10 now creating
Case 11 now creating
Case 12 now creating
Case 13 now creating
Case 14 now creating
Case 15 now creating
Case 16 now creating
Case 17 now creating
Case 18 now creating
Case 19 now creating
Case 20 now creating
Case 21 now creating
Case 22 now creating
Case 23 now creating
Case 24 now creating
Case 25 now creating


0

The script below is running GenX original caserunner. To run this file, uncomment 
run_genx_case!(dirname(@__FILE__), Gurobi.Optimizer) in Run.jl. 


In [15]:

# Determine root_dir
if "__file__" in globals():
    # Running as a script
    script_dir = os.path.dirname(os.path.abspath(__file__))
else:
    script_dir = os.getcwd()

# Traverse up to WEC-DECIDER root
root_dir = os.path.abspath(os.path.join(script_dir, "..", ".."))  
cem_dir = os.path.join(root_dir, "modules", "CEM")
case_folder = "case_1_0"
run_file_path = os.path.join(cem_dir, "Cases", case_folder, "Run.jl")
julia_project_path = cem_dir  

# Confirm the script exists
if not os.path.exists(run_file_path):
    print(f"Run.jl not found at {run_file_path}")
else:
    print(f"Running GenX for case: {case_folder}...\n")

    process = subprocess.Popen(
        ["julia", f"--project={julia_project_path}", run_file_path],
        cwd=root_dir,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True
    )

    # Live stdout printing
    for line in process.stdout:
        print(line, end="")

    stdout, stderr = process.communicate()

    if stderr.strip():
        print("\n Julia stderr:")
        print(stderr)



Running GenX for case: case_1_0...

  ____           __  __   _ _
 / ___| ___ _ __ \ \/ /  (_) |
| |  _ / _ \ '_ \ \  /   | | |
| |_| |  __/ | | |/  \ _ | | |
 \____|\___|_| |_/_/\_(_)/ |_|
                       |__/
 Version: 0.4.4

Configuring Settings
Configuring Solver
Loading Inputs
Reading Input CSV Files
Reading DC-OPF values...

 Julia stderr:
  Activating project at `~/WEC-DECIDER/modules/CEM`
    Building Gurobi → `~/.julia/scratchspaces/44cfe95a-1eb2-52ea-b672-e2afdf69b78f/0af671809e0cde0131a2f8607397197fd3648084/build.log`
┌ Warning: Specifying the network map as a matrix is deprecated as of v0.4
│ and will be removed in v0.5. Instead, use the more compact list-style format.
│ 
│ ..., Network_Lines, Start_Zone, End_Zone, ...
│                  1,          1,        2,
│                  2,          1,        3,
│                  3,          2,        3,
└ @ GenX ~/.julia/packages/GenX/SND5y/src/load_inputs/load_network_data.jl:175
ERROR: LoadError: ArgumentError: column n